In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from atomistic_tools import igor

In [ ]:
import matplotlib

class FormatScalarFormatter(matplotlib.ticker.ScalarFormatter):
    def __init__(self, fformat="%1.1f", offset=True, mathText=True):
        self.fformat = fformat
        matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,
                                                        useMathText=mathText)
    def _set_format(self, vmin, vmax):
        self.format = self.fformat
        if self._useMathText:
            self.format = '$%s$' % matplotlib.ticker._mathdefault(self.format)


def make_plot(fig, ax, data, extent, title=None, title_size=None, center0=False, vmin=None, vmax=None, cmap='gist_heat', noadd=False):
    if center0:
        data_amax = np.max(np.abs(data))
        im = ax.imshow(data.T, origin='lower', cmap=cmap, interpolation='bicubic', extent=extent, vmin=-data_amax, vmax=data_amax)
    else:
        im = ax.imshow(data.T, origin='lower', cmap=cmap, interpolation='bicubic', extent=extent, vmin=vmin, vmax=vmax)
    
    if noadd:
        ax.set_xticks([])
        ax.set_yticks([])
    else:
        ax.set_xlabel(r"x ($\AA$)")
        ax.set_ylabel(r"y ($\AA$)")
        if 1e-3 < np.max(data) < 1e3:
            cb = fig.colorbar(im, ax=ax)
        else:
            cb = fig.colorbar(im, ax=ax, format=FormatScalarFormatter("%.1f"))
        cb.formatter.set_powerlimits((-2, 2))
        cb.update_ticks()
    if title:
        ax.set_title(title)
    if title_size:
        ax.title.set_fontsize(title_size)
    ax.axis('scaled')
    
    
def make_series_plot(fig, data, voltages):
    for i_bias, bias in enumerate(voltages):
        ax = plt.subplot(1, len(voltages), i_bias+1)
        make_plot(fig, ax, data[:, :, i_bias], title="V=%.2f"%bias, title_size=22, cmap='gist_heat', noadd=True)

In [ ]:
npz_path = "/home/kristjan/local_work/test-stm/pyrene_13uc/stm.npz"
loaded_data = np.load(npz_path)

In [ ]:
isovalues = loaded_data['isovalues']
heights = loaded_data['heights']
e_arr = loaded_data['e_arr']
x_arr = loaded_data['x_arr']
y_arr = loaded_data['y_arr']

cc_sts = loaded_data['cc_sts']
cc_stm = loaded_data['cc_stm'].astype(np.float32)
ch_sts = loaded_data['ch_sts']
ch_stm = loaded_data['ch_stm']

In [ ]:
extent = [np.min(x_arr), np.max(x_arr), np.min(y_arr), np.max(y_arr)]

In [ ]:
sets = [cc_stm, cc_sts, ch_stm, ch_sts]
sel_set = [0, 0, 0, 0]

figsize = (12, 3*(extent[3] - extent[2])/(extent[1] - extent[0]))

for i_e, e in enumerate(e_arr):
    fig, ax_arr = plt.subplots(1, 4, figsize=figsize)
    print("E = %.2f eV" % e)
    for i_ax, ax in enumerate(ax_arr):
        make_plot(fig, ax, sets[i_ax][sel_set[i_ax], :, :, i_e], extent, title="", noadd=True)
    plt.show()

# View orbitals

In [ ]:
orb_path = "/home/kristjan/local_work/test-stm/pyrene_13uc/orb.npz"
loaded_orbs = np.load(orb_path)

In [ ]:
loaded_orbs.files

In [ ]:
orbital_data = loaded_orbs['orbitals']
heights = loaded_orbs['heights']
orb_list = loaded_orbs['orb_list']
x_arr = loaded_orbs['x_arr']
y_arr = loaded_orbs['y_arr']

extent = [x_arr[0], x_arr[-1], y_arr[0], y_arr[-1]]

In [ ]:
orb_list

In [ ]:
i_spin = 0
i_height = 0
i_orb = 3

In [ ]:
plt.figure(figsize=(20, 14))
plt.imshow(orbital_data[i_spin][i_height][i_orb].T)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 8))
ax = plt.gca()
make_plot(fig, ax, orbital_data[i_spin][i_height][i_orb]**2, extent, cmap='seismic')
plt.show()

In [ ]:
igorwave = igor.Wave2d(
        data=orbital_data[i_spin][i_height][i_orb]**2,
        xmin=x_arr[0],
        xmax=x_arr[-1],
        xlabel='x [Angstroms]',
        ymin=y_arr[0],
        ymax=y_arr[-1],
        ylabel='y [Angstroms]',
)
igorwave.write("orbital.itx")